<a href="https://colab.research.google.com/github/Lostenergydrink/anime-video-processing-colab/blob/main/Anime_Video_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anime Video Processing Notebook

This notebook provides tools for processing anime videos in two ways:
1. **Upscaling**: Increase resolution from 1080p to 4K
2. **Frame Interpolation**: Increase framerate from 24fps to 60fps/120fps

You can use either or both sections as needed. Each section is self-contained and can be run independently.

**Note**: This notebook uses the free GPU provided by Google Colab. Processing 4K videos or interpolating to high frame rates can take significant time. Consider processing in small chunks for testing.

## 0. Setup and Preparation

### 0.1 Check GPU Availability

First, let's check what GPU we have available. In the free tier, this is usually a T4 or P100.

In [ ]:
!nvidia-smi

### 0.2 Mount Google Drive

We'll mount your Google Drive to store input videos and save the processed results.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directories for our work
!mkdir -p /content/drive/MyDrive/anime_processing/input
!mkdir -p /content/drive/MyDrive/anime_processing/output/upscaled
!mkdir -p /content/drive/MyDrive/anime_processing/output/interpolated

print("\nDirectories created in your Google Drive:")
print("/drive/MyDrive/anime_processing/input - Place your input videos here")
print("/drive/MyDrive/anime_processing/output/upscaled - Upscaled videos will be saved here")
print("/drive/MyDrive/anime_processing/output/interpolated - Interpolated videos will be saved here")

### 0.3 Install Common Dependencies

These packages are needed for both upscaling and interpolation.

In [ ]:
!apt-get update
!apt-get install -y ffmpeg
!pip install -q opencv-python matplotlib tqdm

### 0.4 Helper Functions

These functions will help with file handling and progress display.

In [ ]:
import os
import cv2
import time
import numpy as np
import subprocess
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def get_video_info(video_path):
    """Get information about a video file."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps if fps > 0 else 0
    
    cap.release()
    
    return {
        'width': width,
        'height': height,
        'fps': fps,
        'frame_count': frame_count,
        'duration': duration
    }

def display_video_info(video_path):
    """Display information about a video file in a nice format."""
    info = get_video_info(video_path)
    if info is None:
        print(f"Could not open video: {video_path}")
        return
    
    minutes, seconds = divmod(info['duration'], 60)
    hours, minutes = divmod(minutes, 60)
    
    html = f"""
    <div style="background-color: #f8f9fa; padding: 12px; border-radius: 6px; margin-bottom: 12px">
        <h3 style="margin-top: 0">Video Information</h3>
        <p><b>File:</b> {os.path.basename(video_path)}</p>
        <p><b>Resolution:</b> {info['width']}×{info['height']} pixels</p>
        <p><b>Frame Rate:</b> {info['fps']:.2f} fps</p>
        <p><b>Duration:</b> {int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}</p>
        <p><b>Total Frames:</b> {info['frame_count']}</p>
    </div>
    """
    display(HTML(html))
    return info

def list_input_videos():
    """List all video files in the input directory."""
    input_dir = "/content/drive/MyDrive/anime_processing/input"
    video_extensions = [".mp4", ".mkv", ".avi", ".mov", ".webm"]
    videos = []
    
    for file in os.listdir(input_dir):
        if any(file.lower().endswith(ext) for ext in video_extensions):
            videos.append(os.path.join(input_dir, file))
    
    if not videos:
        print("No videos found in the input directory.")
        print(f"Please upload videos to: {input_dir}")
    else:
        print(f"Found {len(videos)} videos in the input directory:")
        for i, video in enumerate(videos, 1):
            print(f"{i}. {os.path.basename(video)}")
    
    return videos

def extract_frame(video_path, frame_number=0):
    """Extract a single frame from a video for preview."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_number >= frame_count:
        frame_number = frame_count // 2
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame
    else:
        return None

def show_frame(frame, title="Frame Preview"):
    """Display a video frame."""
    plt.figure(figsize=(12, 8))
    plt.title(title)
    plt.imshow(frame)
    plt.axis('off')
    plt.show()

def estimate_processing_time(video_info, is_upscaling=True, scale_factor=2, target_fps=None):
    """Roughly estimate processing time based on video specs."""
    # These are rough estimates based on T4 GPU performance
    base_time_per_frame = 0.15  # seconds per 1080p frame for upscaling
    if not is_upscaling:
        base_time_per_frame = 0.1  # seconds per frame for interpolation
    
    # Scale based on resolution
    resolution_factor = (video_info['width'] * video_info['height']) / (1920 * 1080)
    
    # Adjust for scale factor in upscaling
    if is_upscaling:
        time_multiplier = resolution_factor * (scale_factor ** 2)
        frames_to_process = video_info['frame_count']
    else:
        time_multiplier = resolution_factor
        # For interpolation, estimate new frame count
        new_frames = video_info['frame_count'] * (target_fps / video_info['fps'] - 1)
        frames_to_process = new_frames
    
    # Calculate total processing time
    total_time = frames_to_process * base_time_per_frame * time_multiplier
    
    # Add overhead for I/O and other processing
    total_time *= 1.2
    
    return total_time

def format_time(seconds):
    """Format seconds into hours, minutes, seconds."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours}h {minutes}m {seconds}s"

## 1. Upscaling Module

This section will help you upscale videos from 1080p to higher resolutions (e.g., 4K) while preserving quality and details.

### 1.1 Install Real-ESRGAN

We'll use Real-ESRGAN, which is particularly good for anime content.

In [ ]:
!pip install realesrgan

### 1.2 Download Models

We'll download models specifically trained for anime content.

In [ ]:
!mkdir -p /content/models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P /content/models/
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-general-x4v3.pth -P /content/models/

### 1.3 List Available Input Videos

Let's see what videos are available in the input directory.

In [ ]:
input_videos = list_input_videos()

If you don't see your videos, upload them to `/content/drive/MyDrive/anime_processing/input` in your Google Drive.

### 1.4 Select a Video to Upscale

Choose one of the videos from your input directory.

In [ ]:
# Change this to the number of the video you want to upscale
selected_video_index = 1  # e.g., 1 for the first video in the list

if input_videos and 1 <= selected_video_index <= len(input_videos):
    input_video_path = input_videos[selected_video_index - 1]
    print(f"Selected video: {os.path.basename(input_video_path)}")
    
    # Get and display video info
    video_info = display_video_info(input_video_path)
    
    # Show a preview frame
    preview_frame = extract_frame(input_video_path, frame_number=video_info['frame_count'] // 4)
    if preview_frame is not None:
        show_frame(preview_frame, "Input Video Preview")
else:
    print("Please upload videos to your Google Drive first or select a valid index.")

### 1.5 Upscaling Configuration

Set the parameters for upscaling.

In [ ]:
# Upscaling Settings
upscale_settings = {
    'scale': 2,              # Scale factor: 2 = 2x resolution, 3 = 3x, 4 = 4x
    'model': 'anime',        # 'anime' or 'general'
    'face_enhance': True,    # Enhance face details
    'output_format': 'mp4',  # Output format: mp4, mkv
    'segment_size': 5,       # Minutes per segment (reduce if memory issues)
    'quality': 'high',       # Video quality: 'low', 'medium', 'high'
    'start_time': 0,         # Start time in seconds (0 = beginning)
    'duration': 0            # Duration to process in seconds (0 = entire video)
}

# Quality presets (CRF value, lower = better quality but larger file)
quality_presets = {
    'low': 28,
    'medium': 23,
    'high': 18
}

# Set output filename
if 'input_video_path' in locals():
    video_basename = os.path.splitext(os.path.basename(input_video_path))[0]
    output_filename = f"{video_basename}_upscaled_{upscale_settings['scale']}x.{upscale_settings['output_format']}"
    output_video_path = f"/content/drive/MyDrive/anime_processing/output/upscaled/{output_filename}"
    
    print(f"Output will be saved as: {output_filename}")
    
    # Estimate processing time
    if 'video_info' in locals():
        estimated_time = estimate_processing_time(video_info, is_upscaling=True, scale_factor=upscale_settings['scale'])
        print(f"Estimated processing time: {format_time(estimated_time)}")
        print("(This is a rough estimate. Actual time may vary based on video content and Colab's GPU availability)")

### 1.6 Upscaling Function

This function will handle video upscaling with progress tracking.

In [ ]:
def upscale_video(input_path, output_path, settings):
    """Upscale a video using Real-ESRGAN."""
    # Verify input exists
    if not os.path.exists(input_path):
        print(f"Error: Input file {input_path} does not exist.")
        return False
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Get video info
    video_info = get_video_info(input_path)
    if video_info is None:
        print("Could not read video information.")
        return False
    
    # Determine appropriate model path
    if settings['model'] == 'anime':
        model_path = '/content/models/realesr-animevideov3.pth'
        model_name = 'realesr-animevideov3'
    else:
        model_path = '/content/models/realesr-general-x4v3.pth'
        model_name = 'realesr-general-x4v3'
    
    # If not processing the entire video, extract segment
    working_input = input_path
    if settings['start_time'] > 0 or (settings['duration'] > 0 and settings['duration'] < video_info['duration']):
        working_input = '/content/segment_input.mp4'
        duration_param = f'-t {settings["duration"]}' if settings['duration'] > 0 else ''
        !ffmpeg -y -ss {settings['start_time']} {duration_param} -i "{input_path}" -c:v libx264 -crf 18 -preset fast "{working_input}"
        video_info = get_video_info(working_input)
    
    # If video is large, process in segments
    segment_size_frames = int(settings['segment_size'] * 60 * video_info['fps'])
    if video_info['frame_count'] > segment_size_frames and settings['segment_size'] > 0:
        segment_count = (video_info['frame_count'] + segment_size_frames - 1) // segment_size_frames
        print(f"Processing in {segment_count} segments...")
        
        # Create temporary directory for segments
        !mkdir -p /content/segments
        segments_output = []
        
        for i in range(segment_count):
            start_frame = i * segment_size_frames
            start_time = start_frame / video_info['fps']
            duration = min(settings['segment_size'] * 60, video_info['duration'] - start_time)
            
            print(f"\nProcessing segment {i+1}/{segment_count} (from {format_time(start_time)})")
            
            # Extract segment
            segment_input = f"/content/segments/segment_{i:03d}_input.mp4"
            !ffmpeg -y -ss {start_time} -t {duration} -i "{working_input}" -c:v libx264 -crf 18 -preset fast "{segment_input}"
            
            # Process segment
            segment_output = f"/content/segments/segment_{i:03d}_output.{settings['output_format']}"
            face_enhance = '--face_enhance' if settings['face_enhance'] else ''
            crf = quality_presets[settings['quality']]
            
            !python -m realesrgan.inference_realesrgan_video \
                --input "{segment_input}" \
                --output "{segment_output}" \
                --model_path "{model_path}" \
                --netscale {settings['scale']} \
                --outscale {settings['scale']} \
                --suffix "" \
                --model_name "{model_name}" \
                --fps "{video_info['fps']}" \
                --ffmpeg_crf {crf} \
                --preset slower \
                {face_enhance}
            
            segments_output.append(segment_output)
        
        # Combine segments
        print("\nCombining segments...")
        segments_list = '/content/segments_list.txt'
        with open(segments_list, 'w') as f:
            for segment in segments_output:
                f.write(f"file '{segment}'\n")
        
        !ffmpeg -y -f concat -safe 0 -i "{segments_list}" -c copy "{output_path}"
        
        # Clean up segments
        print("Cleaning up temporary files...")
        !rm -rf /content/segments
        !rm -f "{segments_list}"
    else:
        # Process entire video at once
        face_enhance = '--face_enhance' if settings['face_enhance'] else ''
        crf = quality_presets[settings['quality']]
        
        !python -m realesrgan.inference_realesrgan_video \
            --input "{working_input}" \
            --output "{output_path}" \
            --model_path "{model_path}" \
            --netscale {settings['scale']} \
            --outscale {settings['scale']} \
            --suffix "" \
            --model_name "{model_name}" \
            --fps "{video_info['fps']}" \
            --ffmpeg_crf {crf} \
            --preset slower \
            {face_enhance}
    
    # Clean up temporary files
    if working_input != input_path:
        !rm -f "{working_input}"
    
    # Verify output was created
    if os.path.exists(output_path):
        output_info = get_video_info(output_path)
        if output_info:
            print(f"\nUpscaling complete! Output saved to: {output_path}")
            print(f"Original resolution: {video_info['width']}x{video_info['height']}")
            print(f"New resolution: {output_info['width']}x{output_info['height']}")
            return True
    
    print("Error: Failed to create output file.")
    return False

### 1.7 Run Upscaling

Now let's upscale the selected video.

In [ ]:
# Only run if we have selected a valid video
if 'input_video_path' in locals() and 'output_video_path' in locals():
    print("Starting upscaling process...")
    start_time = time.time()
    
    success = upscale_video(input_video_path, output_video_path, upscale_settings)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    if success:
        print(f"Total processing time: {format_time(elapsed_time)}")
        
        # Display a frame from the upscaled video
        upscaled_frame = extract_frame(output_video_path, frame_number=get_video_info(output_video_path)['frame_count'] // 4)
        if upscaled_frame is not None:
            show_frame(upscaled_frame, "Upscaled Video Preview")
    else:
        print("Upscaling failed.")
else:
    print("Please select a valid input video first.")

## 2. Frame Interpolation Module - See Part 2 of the Notebook

Please continue to Part 2 for the frame interpolation module.